In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import nltk
#import datasets
from tqdm import tqdm
import torch
import time
import spacy
from nltk.corpus import stopwords
nltk.download("stopwords")
from heapq import nlargest

from transformers import pipeline
from transformers import BartTokenizer, BartForConditionalGeneration

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/niclascramer/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# NOT RUNNING CODE

In [2]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')

In [3]:
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

In [33]:
filename = 'input/AFD_Wahlprogramm_2021.txt'

In [34]:
with open(filename, encoding="utf8") as f:
    text = f.read()
    #text = text.replace(r'\\d+\\n', '$')
    #text=   re.sub("\n\n", ".", text)
    #text=   re.sub("\d+.", ".", text)
    text=   re.sub(" \d+\n", ".", text)
    text=   re.sub("\n\d+", " ", text)
    text=   re.sub("\n", " ", text)
sentences = re.split(r' *[\.\?!][\'"\)\]]* *', text)

In [63]:
def summarize_text(input_text: list, percentage):
    spacy_model = spacy.load("de_core_news_sm")
    german_stop_words = stopwords.words('german')
    text_list = [word.lower() for word in input_text if word not in german_stop_words and word not in [",", ".", " ", ":", "'", '"']]

    #print(text_list)

    text = spacy_model(list_to_string(text_list))
    tokens  = [token.text for token in text]
    freq_words = {}

    for word in text:
        if word not in freq_words.keys():
            freq_words[word.text] = 1
        else:
            freq_words[word.text] += 1

    max_freq = max(freq_words.values())

    for word in freq_words.keys():
        freq_words[word] = freq_words[word]/max_freq

    print(freq_words)
    
    sentence_tokens = [sent for sent in text.sents]
    sentence_scores = {}

    for sent in sentence_tokens:
        for word in sent:
            if word in freq_words.keys():
                if sent not in sentence_scores.keys():
                    sentence_scores[sent] = freq_words[word.text]
                else:
                    sentence_scores[sent] += freq_words[word.text]
    
    #print(sentence_scores)
    
    select_length = int(len(sentence_tokens)*percentage)
    result = nlargest(select_length, sentence_scores, key=sentence_scores.get)
    final_summary = [word.text for word in result]
    sum =  ''.join(final_summary)
    return sum

In [64]:
sum = summarize_text(sentences, 0.05)
sum

{'deutschland': 1.0, '.': 1.0, 'aber': 1.0, 'normal': 1.0, 'programm': 1.0, 'der': 1.0, 'alternative': 1.0, 'für': 1.0, 'die': 1.0, 'wahl': 1.0, 'zum': 1.0, '20.': 1.0, 'deutschen': 1.0, 'bundestag': 1.0, ' ': 1.0, 'ad': 1.0, 'inhaltsverzeichnis': 1.0, 'demokratie': 1.0, 'und': 1.0, 'rechtsstaat': 1.0, 'das': 1.0, 'volk': 1.0, 'ist': 1.0, 'souverän': 1.0, 'volksabstimmungen': 1.0, 'nach': 1.0, 'schweizer': 1.0, 'modell': 1.0, 'gewaltenteilung': 1.0, 'gewährleisten': 1.0, 'entpolitisierung': 1.0, 'justiz': 1.0, 'verfassungsschutz': 1.0, 'reformieren': 1.0, 'trennung': 1.0, 'von': 1.0, 'amt': 1.0, 'mandat': 1.0, 'kampf': 1.0, 'ämterpatronage': 1.0, 'macht': 1.0, 'parteien': 1.0, 'beschränken': 1.0, 'freie': 1.0, 'listenwahl': 1.0, 'freies': 1.0, 'verkleinerung': 1.0, 'parlamente': 1.0, 'berufspolitikertum': 1.0, ':': 1.0, 'mandats-': 1.0, 'amtszeiten': 1.0, 'begrenzen': 1.0, 'lobbyismus': 1.0, 'eindämmen': 1.0, 'freiheit': 1.0, 'verantwortung': 1.0, 'bürgerlich-freiheitliche': 1.0, 'rech

''

In [11]:
sentences = sentences[:len(sentences)//8]
len(sentences)

32

In [12]:
from transformers import BigBirdPegasusForConditionalGeneration, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google/bigbird-pegasus-large-pubmed")
model = BigBirdPegasusForConditionalGeneration.from_pretrained("google/bigbird-pegasus-large-pubmed")

In [13]:
print("Inputs")
inputs = tokenizer(sentences, return_tensors='pt', padding=True)

Inputs


In [15]:
print("Predictions Model")
start_time = time.time()
prediction = model.generate(**inputs)
end_time = time.time()
print("Zeit: " + end_time-start_time)

Attention type 'block_sparse' is not possible if sequence_length: 30 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size + config.num_random_blocks * config.block_size + additional buffer: config.num_random_blocks * config.block_size = 704 with config.block_size = 64, config.num_random_blocks = 3. Changing attention type to 'original_full'...


Predictions Model


c:\Users\nikla\anaconda3\envs\NLP-New\lib\site-packages\transformers\generation_utils.py:1359: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 256 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
print("Predictions Batch Decode")
prediction = tokenizer.batch_decode(prediction)

# RUNNING CODE

In [17]:
def summarize_text(text, sentence_count, ignore_sentences=[]):

    from sumy.parsers.plaintext import PlaintextParser
    from sumy.nlp.tokenizers import Tokenizer
    from sumy.summarizers.text_rank import TextRankSummarizer
    import re

    summarizer = TextRankSummarizer()
 
    def change_text(text, additionals=[]):
        text = re.sub(" \d+\n", ".", text)
        text = re.sub("\n\d+", " ", text)
        text = re.sub("\n", " ", text)
        text = re.sub("- ", "", text)
        text = re.sub("[0-9]", "", text)
        text = re.sub("()", "", text)
        for entry in additionals:
            text = re.sub(entry, "", text)
        sentences = re.split(r' *[\.\?!][\'"\)\]]* *', text)
        return sentences
    
    def list_to_string(liste):
        sentence_string = ""
        for elem in liste:
            sentence_string += elem
            sentence_string += ". "
        return sentence_string

    changed_text = change_text(text, ignore_sentences)
    parser = PlaintextParser.from_string(list_to_string(changed_text), Tokenizer("german"))
    summary = summarizer(parser.document, sentence_count)
    text_summary = ""
    for sentence in summary:
        text_summary += str(sentence)
        print(sentence)
        print("----------------")
    return text_summary
    